In [1]:
import pandas as pd
from pandas import DataFrame
from pprint import pprint
from collections import Counter
import math    
df_tennis=pd.DataFrame(data=pd.read_csv('weather.csv'))
print(df_tennis)

def entropy(probs):
    
    return sum([-prob*math.log(prob,2) for prob in probs])
def entropy_of_list(a_list):
   
    cnt=Counter(x for x in a_list)
    num_instances=len(a_list)*1.0
    probs=[x/num_instances for x in cnt.values()]
    return entropy(probs)
total_entropy=entropy_of_list(df_tennis['play'])
print(total_entropy)
def information_gain(df,split_attributes_name,target_attribute_name,trace=0):
    df_split=df.groupby(split_attributes_name)
    nobs=len(df.index)*1.0
    df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list,lambda x:len(x)/nobs]})[target_attribute_name]
    df_agg_ent.columns=['Entropy','PropObservations']
    new_entropy=sum(df_agg_ent['Entropy'] * df_agg_ent['PropObservations'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy
def id3(df,target_attribute_name,attribute_names,default_class=None):
    #from collections import Counter
    cnt=Counter(x for x in df[target_attribute_name])
    print(cnt)
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class=max(cnt.keys())
        gainz=[information_gain(df,attr,target_attribute_name) for attr in attribute_names]
        index_of_max=gainz.index(max(gainz))
        best_attr=attribute_names[index_of_max]
        tree={best_attr:{}}
        remaining_attribute_names=[i for i in attribute_names if i!=best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            subtree=id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val]=subtree
        return tree
a=list(df_tennis.columns)
a.remove('play')
print(a)
tree=id3(df_tennis,'play',a)
print("\n\n The Result Decision Tree: \n")
pprint(tree)

     outlook temperature humidity    wind play
0      sunny         hot     high    weak   no
1      sunny         hot     high  strong   no
2   overcast         hot     high    weak  yes
3      rainy        mild     high    weak  yes
4      rainy        cool   normal    weak  yes
5      rainy        cool   normal  strong   no
6   overcast        cool   normal  strong  yes
7      sunny        mild     high    weak   no
8      sunny        cool   normal    weak  yes
9      rainy        mild   normal    weak  yes
10     sunny        mild   normal  strong  yes
11  overcast        mild     high  strong  yes
12  overcast         hot   normal    weak  yes
13     rainy        mild     high  strong   no
0.9402859586706309
['outlook', 'temperature', 'humidity', 'wind']
Counter({'yes': 9, 'no': 5})
Counter({'yes': 4})
Counter({'yes': 3, 'no': 2})
Counter({'no': 2})
Counter({'yes': 3})
Counter({'no': 3, 'yes': 2})
Counter({'no': 3})
Counter({'yes': 2})


 The Result Decision Tree: 

{'outlook': {